In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
import glob
import os


from ipywidgets import VBox,HBox, interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot

from src.spectrum import spectrum
from src.spec_plot import spec_plot
from src.lcurve_plot import lcurve_plot
from src.mycolors import mycolors
from src.background_models import bg3D

output_notebook()

filelist = [os.path.basename(file) for file in glob.glob(os.path.join('data','raw','*.DTA'))]
file_picker=widgets.Dropdown(description= 'Choose File:', options= filelist)
file_picker

Loading BokehJS ...

Dropdown(description='Choose File:', options=('example_2dDEER.DTA',), value='example_2dDEER.DTA')

In [2]:
spec = spectrum(os.path.join('data','raw',file_picker.value))


#Plot definitions
xs= [spec.raw_time]*2
ys = [spec.real2D[0],spec.imaginary2D[0]]

spec2D = spec_plot(xs,ys,y_range = (spec.imaginary2D.min(),spec.real2D.max()*1.02), plot_height=300)

ys = [spec.real,spec.imaginary]
    
specavg = spec_plot(xs,ys, y_range = (spec.imaginary.min(),spec.real.max()*1.02), plot_height=300) 


#Widget Defintions

def on_button_clicked(val):
    if np.sum(spec.slice_mask)>1: #prevent deleting of all slices
        spec.delete_slice(myslider.value)
        myslider.options=valuelist[spec.slice_mask]
        myslider.value = myslider.options[-1]
        spec.avg_slices()
        update(myslider.value)
    
def update(index):
    for idx,line in enumerate(spec2D.renderers):
        line.data_source.data['y'] = [spec.real2D,spec.imaginary2D][idx][index]
        
    for idx,line in enumerate(specavg.renderers):
        line.data_source.data['y'] = [spec.real,spec.imaginary][idx]
        
    push_notebook(handle=sliceplots)
    return index

valuelist=np.arange(spec.real2D.shape[0])

myslider=widgets.SelectionSlider(description='value', value=valuelist[-1],options=valuelist)
deletebutton=widgets.Button(description='Delete Slice')
deletebutton.on_click(on_button_clicked)



#Render plots, widgets
sliceplots=show(row(spec2D,specavg),notebook_handle=True)
HBox([interactive(update,index=myslider),deletebutton])


In [3]:
cutoffx=[spec.raw_time.max(),spec.raw_time.max()]
cutoffy=[-.1,1]

cutoffslider=widgets.SelectionSlider(description='Cutoff', value=spec.raw_time[-1], options=spec.raw_time)
signal=spec.real+1j*spec.imaginary
forphasing=signal[int(len(signal)/8):list(spec.raw_time).index(cutoffslider.value)]
spec.phase=optimize.fmin(lambda x:abs((forphasing*np.e**(1j*x)).imag.mean()),1,disp=False)[0]
phaseslider=widgets.FloatSlider(description='Phase', value=spec.phase, min=-np.pi, max=np.pi, step=0.001)


xs = [spec.raw_time]*2
ys = [np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase))),
      np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))]


phaseplot=spec_plot(xs,ys)
phaseplot.line(cutoffx,cutoffy, color=(255,0,0), line_width=1) #decorate plot with cutoff line
phaser=show(phaseplot, notebook_handle=True);


phasebutton=widgets.Button(description='Phase')
kernelbutton=widgets.Button(description='Build Kernel')

def phase_button_clicked(val):
    forphasing=signal[int(len(signal)/8):list(spec.raw_time).index(cutoffslider.value)]
    spec.phase=optimize.fmin(lambda x:abs((forphasing*np.e**(1j*x)).imag.mean()),1,disp=False)[0]
    phaseslider.value=spec.phase
    
def kernel_button_clicked(val):
    
    spec.tmax = cutoffslider.value
    spec.cutoff=cutoffslider.value # is cutoff needed (and not just as spec.tmax) ??
    spec.build_kernel()
    spec.real=np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
    spec.imag=np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
    
   

    
def updatePhase(value):
    phaseplot.renderers[-3].data_source.data['y'] = np.real(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))
    phaseplot.renderers[-2].data_source.data['y'] = np.imag(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))
    phaseplot.renderers[-1].data_source.data['y'] = [np.min(np.imag(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))),1]
    push_notebook(handle=phaser)
    return value

def updateCutoff(value):
    phaseplot.renderers[-1].data_source.data['x'] = [value,value]
    push_notebook(handle=phaser)
    return value

#kernel,spec.t,spec.r,Lmatrix,spec.cutoff = 

phasebutton.on_click(phase_button_clicked)
kernelbutton.on_click(kernel_button_clicked)
HBox([interactive(updateCutoff,value=cutoffslider),interactive(updatePhase,value=phaseslider),phasebutton,kernelbutton])

In [10]:
#initial fit, finding spectrum start
#need to reimplement different types of background models

def gaussian(x,r,s,a):
    return a*np.exp(-((x- r)**2/(2*s**2)))

symmetry_point=np.argmax(spec.raw_time>=-spec.raw_time[0])+1 #find index of symmetry point about t=0,+1 for non inclusive
spec.zeropoint,__,spec.zeroamp = gaussfit = optimize.curve_fit(gaussian,
                                                               spec.raw_time[1:symmetry_point],
                                                               spec.real[1:symmetry_point])[0]
spec.update_ranges()
spec.fit_background(bg3D,spec.background_range)
spec.background_correct()

#zeropeak plot

xs = [spec.raw_time[1:symmetry_point]]*2
ys = [spec.real[1:symmetry_point],
      gaussian(spec.raw_time,*gaussfit)[1:symmetry_point]]

zeropointx = [spec.zeropoint]*2
zeropointy = [spec.real[1:symmetry_point].min(),1]

zeropeak = spec_plot(xs,ys, plot_height=300)
zeropeak.line(zeropointx, zeropointy, color=(0,255,0))
zeropeak.circle([spec.zeropoint], [spec.zeroamp], color=(255,0,0))

#bg fit plot

xs = [spec.raw_time]*2
ys = [spec.real,spec.background]

cutoffx=[spec.cutoff]*2
cutoffy=[spec.real.min(),1]

backgroundx=[spec.background_start]*2
backgroundy=[spec.real.min(),1]

bgsub=spec_plot(xs,ys,plot_height=300)
bgsub.line(cutoffx,cutoffy,color=(255,0,0))
bgsub.line(backgroundx,backgroundy,color=(0,0,255))

#waveform plot 
waveformplot=spec_plot(spec.t,spec.waveform,plot_height=300)


processingplots = show(gridplot([[zeropeak,bgsub],[None,waveformplot]]),notebook_handle=True)

windowslider=widgets.IntSlider(description='Fit window', 
                               value=symmetry_point, min=4+np.argmax(spec.real==spec.real.max()), max=round(len(spec.raw_time)/30))
bgslider=widgets.SelectionSlider(description='Background Start', 
                                 value=spec.background_start, 
                                 options=spec.raw_time[4+np.argmax(spec.real==spec.real.max()):np.argmax(spec.raw_time==spec.cutoff)-20])
normalizer=widgets.Dropdown(description= 'V(t=0)', options=["Gaussian Fit",
                                   "Gaussian Center, Data Maximum Amp", 
                                   "Data Max Center, Data Maximum Amp"])

def updateBg(value): 
    spec.background_start = value
    spec.update_ranges()

    spec.fit_background(bg3D,spec.background_range)
    bgsub.renderers[-3].data_source.data['y'] = spec.background
    bgsub.renderers[-1].data_source.data['x']=[value]*2
    
    spec.background_correct()
    
    waveformplot.renderers[-1].data_source.data={'x':spec.t,'y':spec.waveform}
    
    push_notebook(handle=processingplots)


def updateWindow(value):
    spec.zeropoint,__,spec.zeroamp = gaussfit = optimize.curve_fit(gaussian,
                                                                   spec.raw_time[1:value],
                                                                   spec.real[1:value])[0]

    if normalizer.value == "Gaussian Center, Data Maximum Amp":
        spec.zeroamp = spec.real[abs(spec.raw_time-spec.zeropoint).argmin()]
    elif normalizer.value == "Data Max Center, Data Maximum Amp":
        spec.zeropoint = spec.raw_time[np.argmax(spec.real==spec.real.max())]
        spec.zeroamp = spec.real.max()
        
    spec.update_ranges()
    updateBg(bgslider.value)
    
    newx=spec.raw_time[1:value]
    newy=spec.real[1:value]
    fity=gaussian(newx,*gaussfit)
    plot_order = [[newx,newy],
                  [newx,fity],
                  [[spec.zeropoint]*2,[newy.min(),1]],
                  [[spec.zeropoint],[spec.zeroamp]]]
    
    for idx, line in enumerate(zeropeak.renderers):
        line.data_source.data = {'x': plot_order[idx][0], 'y': plot_order[idx][1]}

    
    push_notebook(handle=processingplots)
    

def updateNorm(value):
    normMethod = value
    updateWindow(windowslider.value)
    updateBg(bgslider.value)

VBox([HBox([interactive(updateWindow,value=windowslider),interactive(updateNorm,value=normalizer)]),
HBox([interactive(updateBg,value=bgslider)])])

In [11]:
spec.generate_lcurve()

#l-curve compoenents, initialize choice of alpha with LOOCV
residualnorm=np.linalg.norm(spec.tikhonovfits-spec.waveform,axis=1)**2
solutionnorm=np.linalg.norm(np.dot(spec.Lmatrix,spec.solutions.T).T,axis=1)**2
alpha_LOOCV_index=(residualnorm/(np.sum(1-spec.hMats,axis=1))**2).argmin()


#l-curve
lcurve = lcurve_plot([residualnorm,solutionnorm],plot_height=300)
lcurve.circle([residualnorm[alpha_LOOCV_index]], [solutionnorm[alpha_LOOCV_index]],color=(255,0,0)) #LOOCV choice for alpha
lcurve.circle([residualnorm[alpha_LOOCV_index]], [solutionnorm[alpha_LOOCV_index]],color=(0,0,0)) #user defined choice for alpha

#waveform fit
xs = [spec.t]*2
ys = [spec.waveform,spec.tikhonovfits[alpha_LOOCV_index]]

tikresults=spec_plot(xs,ys,plot_height=300)

#fit residual
tikresidual=spec_plot(spec.t,spec.waveform-spec.tikhonovfits[alpha_LOOCV_index],
                      plot_height=300,
                      y_range=([(spec.waveform-spec.tikhonovfits[alpha_LOOCV_index]).min(),
                                (spec.waveform-spec.tikhonovfits[alpha_LOOCV_index]).max()]))

#Distance distributions

xs = [spec.r]*2
ys = [spec.solutions[alpha_LOOCV_index]]*2
distances = spec_plot(xs,ys,plot_height = 300,colorlist = [(255,0,0),mycolors(0)])
distances.xaxis.axis_label="Distance (nm)"
# show the results


lcurveplots = show(gridplot([[lcurve,tikresidual],[tikresults,distances]]),notebook_handle=True)

def updateAlpha(value):
    spec.alpha=spec.alphas[value]
    tikresidual.renderers[-1].data_source.data['y']=spec.waveform-spec.tikhonovfits[value]
    tikresults.renderers[-1].data_source.data['y']=spec.tikhonovfits[value]
    lcurve.renderers[-1].data_source.data={'x':[residualnorm[value]],'y':[solutionnorm[value]]}
    distances.renderers[-1].data_source.data['y'] = spec.solutions[value]

    push_notebook(handle=lcurveplots)

#slider for index of list of alphas, actual alpha value can be found with spec.alpha
alphaslider=widgets.IntSlider(description='Alpha Value', value=alpha_LOOCV_index, min=0, max=len(spec.alphas)-1)
HBox([interactive(updateAlpha,value=alphaslider)])

In [12]:
##BG VALIDATION

spec.validate_background()

backgrounds_plot = spec_plot(spec.raw_time+abs(0*spec.backgrounds),spec.backgrounds, plot_height=300)
backgrounds_plot.line(spec.raw_time,spec.real)

spectrums_plot = spec_plot(spec.t+abs(0*spec.waveforms),spec.waveforms, plot_height=300) #This is a DDT Lab joke

show(row(backgrounds_plot, spectrums_plot))